# COVID-19 myDense CNN training
This notebook is a direct copy of the one used in google colab and should not be run localy. 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/ERSCOVIDTest/COVID-19/')

In [4]:
from myDense import myDenseNetv2Edited
from myDense import myDenseNetv2
from myDense import myDenseNetv2Dropout
import pandas as pd
import scipy.io
import numpy as np
import csv
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from tensorflow.keras.metrics import AUC,Accuracy,Precision,Recall,SensitivityAtSpecificity

def import_data(t):
        t = '/content/drive/My Drive/COVID-19 CT dataset/' + t
        files = []
        positive = []
        with open(t + '.csv', newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
            for row in spamreader:
                positive.append(row[0][-24])
                row = t + '/' + row[0]
                files.append(row)
        return files, positive


def load_data(dataset):

    print(f"Loading {dataset} data")
    train, Y_train = import_data(dataset)
    print(f"Data loaded. {len(Y_train)} samples")

    Y_train = np.array(Y_train)
    Y_train = np.where(Y_train == '+', 1, Y_train)
    Y_train = np.where(Y_train == '-', 0, Y_train)
    Y_train = Y_train.astype(np.float)

    X_train = []
    for fileName in train:
        mat = scipy.io.loadmat(fileName)
        X_train.append(mat['only_lung_zoomed_3std'])
    X_train = np.array(X_train)
    X_train = X_train[..., np.newaxis]
    X_train = np.swapaxes(X_train, 1, 3)
    X_train = np.swapaxes(X_train, 2, 3)
    return X_train,Y_train

In [7]:
X_train,Y_train = load_data('training')

Loading training data
Data loaded. 86 samples


In [8]:
X_test,Y_test = load_data('test')

Loading test data
Data loaded. 24 samples


In [9]:
X_validate,Y_validate = load_data('validation')

Loading validation data
Data loaded. 12 samples


## Defult mode

In [10]:
model_normal = myDenseNetv2((48, 240, 360, 1))
model_normal.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',AUC(),Precision(),Recall()])

model_normal.fit(X_train,Y_train, validation_data=(X_validate,Y_validate),epochs=10,verbose=1,batch_size=1)

Epoch 1/10
86/86 [==============================] - 92s 608ms/step - loss: 0.8156 - accuracy: 0.4894 - auc: 0.3485 - precision: 0.5694 - recall: 0.6764 - val_loss: 35.2613 - val_accuracy: 0.5833 - val_auc: 0.5000 - val_precision: 0.5833 - val_recall: 1.0000
Epoch 2/10
86/86 [==============================] - 54s 627ms/step - loss: 0.7037 - accuracy: 0.5462 - auc: 0.5264 - precision: 0.5579 - recall: 0.7662 - val_loss: 0.6982 - val_accuracy: 0.5000 - val_auc: 0.5571 - val_precision: 0.6000 - val_recall: 0.4286
Epoch 3/10
86/86 [==============================] - 55s 634ms/step - loss: 0.6642 - accuracy: 0.6405 - auc: 0.6335 - precision: 0.5461 - recall: 0.4943 - val_loss: 1.7346 - val_accuracy: 0.4167 - val_auc: 0.4857 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/10
86/86 [==============================] - 54s 634ms/step - loss: 0.7015 - accuracy: 0.5141 - auc: 0.5687 - precision: 0.5156 - recall: 0.6564 - val_loss: 1.2700 - val_accuracy: 0.5833 - val_auc: 0.1857 - val_pr

In [ ]:
y_pred = model_normal.predict(X_test, batch_size=1,verbose=1)

24/24 [==============================] - 3s 117ms/step


## Added L1 regularization
With $\lambda$ = 0.01

In [11]:
model_l1 = myDenseNetv2Edited((48, 240, 360, 1), "l1",0.01)
model_l1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',AUC(),Precision(),Recall()])

model_l1.fit(X_train,Y_train, validation_data=(X_validate,Y_validate),epochs=10,verbose=1,batch_size=1)

Epoch 1/10
86/86 [==============================] - 62s 655ms/step - loss: 117.8140 - accuracy: 0.4967 - auc_1: 0.4052 - precision_1: 0.4620 - recall_1: 0.4577 - val_loss: 15.0277 - val_accuracy: 0.4167 - val_auc_1: 0.5000 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/10
86/86 [==============================] - 54s 629ms/step - loss: 12.4182 - accuracy: 0.5514 - auc_1: 0.3941 - precision_1: 0.5701 - recall_1: 0.7957 - val_loss: 8.5617 - val_accuracy: 0.4167 - val_auc_1: 0.5000 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/10
86/86 [==============================] - 54s 634ms/step - loss: 7.4111 - accuracy: 0.5498 - auc_1: 0.3402 - precision_1: 0.5826 - recall_1: 0.8351 - val_loss: 6.0836 - val_accuracy: 0.4167 - val_auc_1: 0.5000 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 4/10
86/86 [==============================] - 54s 634ms/step - loss: 5.4516 - accuracy: 0.4578 - auc_1: 0.3479 - precision_1: 0.4304 - recall_1: 0.4843 - val_l

## Added L2 regularization
With $\lambda$ = 0.01

In [12]:
model_l2 = myDenseNetv2Edited((48, 240, 360, 1), "l2",0.01)
model_l2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',AUC(),Precision(),Recall()])

model_l2.fit(X_train,Y_train, validation_data=(X_validate,Y_validate),epochs=10,verbose=1,batch_size=1)

Epoch 1/10
86/86 [==============================] - 62s 654ms/step - loss: 7.0530 - accuracy: 0.5713 - auc_2: 0.4367 - precision_2: 0.5862 - recall_2: 0.7582 - val_loss: 2.9469 - val_accuracy: 0.5833 - val_auc_2: 0.4000 - val_precision_2: 0.5833 - val_recall_2: 1.0000
Epoch 2/10
86/86 [==============================] - 54s 631ms/step - loss: 2.5884 - accuracy: 0.5213 - auc_2: 0.5732 - precision_2: 0.5346 - recall_2: 0.5941 - val_loss: 1.9371 - val_accuracy: 0.3333 - val_auc_2: 0.1429 - val_precision_2: 0.3333 - val_recall_2: 0.1429
Epoch 3/10
86/86 [==============================] - 54s 632ms/step - loss: 1.8614 - accuracy: 0.4792 - auc_2: 0.4297 - precision_2: 0.4921 - recall_2: 0.4127 - val_loss: 8.5449 - val_accuracy: 0.5833 - val_auc_2: 0.5000 - val_precision_2: 0.5833 - val_recall_2: 1.0000
Epoch 4/10
86/86 [==============================] - 54s 632ms/step - loss: 1.5280 - accuracy: 0.6212 - auc_2: 0.4583 - precision_2: 0.6300 - recall_2: 0.8640 - val_loss: 13.9710 - val_accuracy:

## Adding Dropout


In [13]:
model_dropout = myDenseNetv2Dropout((48, 240, 360, 1), 0.3)
model_dropout.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',AUC(),Precision(),Recall()])

model_dropout.fit(X_train,Y_train, validation_data=(X_validate,Y_validate),epochs=10,verbose=1,batch_size=1)

Epoch 1/10
86/86 [==============================] - 61s 651ms/step - loss: 0.7264 - accuracy: 0.5164 - auc_3: 0.4702 - precision_3: 0.5445 - recall_3: 0.5867 - val_loss: 0.7091 - val_accuracy: 0.4167 - val_auc_3: 0.5571 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00
Epoch 2/10
86/86 [==============================] - 54s 633ms/step - loss: 0.7173 - accuracy: 0.4789 - auc_3: 0.3965 - precision_3: 0.5339 - recall_3: 0.7074 - val_loss: 24.9618 - val_accuracy: 0.5833 - val_auc_3: 0.5000 - val_precision_3: 0.5833 - val_recall_3: 1.0000
Epoch 3/10
86/86 [==============================] - 54s 633ms/step - loss: 0.6610 - accuracy: 0.6563 - auc_3: 0.5427 - precision_3: 0.6568 - recall_3: 0.9484 - val_loss: 11.9775 - val_accuracy: 0.4167 - val_auc_3: 0.5000 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00
Epoch 4/10
86/86 [==============================] - 54s 632ms/step - loss: 0.6635 - accuracy: 0.6340 - auc_3: 0.6081 - precision_3: 0.7866 - recall_3: 0.3979 - val_loss: 4.536

In [5]:
def score_models(X_test,Y_test, models = []):
    for model in models:
        # score every model iwth every metric
        results = model.evaluate(X_test, Y_test, batch_size=1)

In [14]:
score_models(X_test,Y_test, models=[model_normal, model_l1,model_l2,model_dropout])

24/24 [==============================] - 3s 126ms/step - loss: 1.5713 - accuracy: 0.5000 - auc_3: 0.4536 - precision_3: 0.5455 - recall_3: 0.8571


## Summary
No real improvement was observed, this is most likely due to the fact, that our dataset is to small 2% of oryginal dataset (about 40% of maximum we found), and our model is uderfitted not overfitted. 